In [1]:
import pandas as pd

In [2]:
test_data = pd.read_csv('Problem 1/last_orders_subset.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5487 entries, 0 to 5486
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          5487 non-null   int64 
 1   SKU            5487 non-null   int64 
 2   Member         5487 non-null   object
 3   Delivery Date  5487 non-null   object
 4   Name           5487 non-null   object
dtypes: int64(2), object(3)
memory usage: 214.5+ KB


In [3]:
order_list = list(set(test_data["Order"]))
len(order_list)

638

In [4]:
df_list = []
for order in order_list:
    top5 = test_data[test_data["Order"] == order].head()
    df_list.append(top5)
sample_submission = pd.concat(df_list, ignore_index = True)

In [5]:
sample_submission

,Order,SKU,Member,Delivery Date,Name
0,7817216,15669994,SWNHNRN,03/09/12,Other Dals
1,7817216,15668465,SWNHNRN,03/09/12,Root Vegetables
2,7817216,15668687,SWNHNRN,03/09/12,Root Vegetables
3,7817216,15668468,SWNHNRN,03/09/12,Beans
4,7817216,15668520,SWNHNRN,03/09/12,Bread
...,...,...,...,...,...
3185,7559166,15668685,SWOOOLL,29/06/13,Beans
3186,7559166,15668469,SWOOOLL,29/06/13,Beans
3187,7559166,15668465,SWOOOLL,29/06/13,Root Vegetables
3188,7559166,15668462,SWOOOLL,29/06/13,Gourd & Cucumber


In [6]:
sample_submission = sample_submission[["Order", "SKU", "Member"]].reset_index().rename(columns = {"index": "ID"})
sample_submission["ID"] = sample_submission["ID"].apply(lambda x: x+1)
sample_submission

,ID,Order,SKU,Member
0,1,7817216,15669994,SWNHNRN
1,2,7817216,15668465,SWNHNRN
2,3,7817216,15668687,SWNHNRN
3,4,7817216,15668468,SWNHNRN
4,5,7817216,15668520,SWNHNRN
...,...,...,...,...
3185,3186,7559166,15668685,SWOOOLL
3186,3187,7559166,15668469,SWOOOLL
3187,3188,7559166,15668465,SWOOOLL
3188,3189,7559166,15668462,SWOOOLL


In [7]:
sample_submission.to_csv("sample_submission.csv")

# Baseline Recommendation

Based on the frequency of the items in the past orders 

In [8]:
df = pd.read_csv('Problem 1/all_except_last_orders.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28984 entries, 0 to 28983
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          28984 non-null  int64 
 1   SKU            28984 non-null  int64 
 2   Member         28984 non-null  object
 3   Delivery Date  28984 non-null  object
 4   Name           28984 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


In [9]:
sku_freq_df = df[["Member", "SKU"]].groupby(["Member", "SKU"]) \
                .size() \
                .reset_index(name='frequency') \
                .sort_values(by = ["Member", "frequency"], ascending = [True, False])

sku_freq_df

,Member,SKU,frequency
2,SSCEHNS,7580823,7
8,SSCEHNS,15668377,7
34,SSCEHNS,15669865,7
20,SSCEHNS,15669772,6
40,SSCEHNS,15669970,6
...,...,...,...
18387,SWRNHCS,15669874,1
18388,SWRNHCS,15669886,1
18389,SWRNHCS,15670196,1
18390,SWRNHCS,15670260,1


In [10]:
test_data

,Order,SKU,Member,Delivery Date,Name
0,7409204,15669778,SWLCNOE,05/09/13,Other Dals
1,8076206,15669977,SWOEZES,01/04/14,Almonds
2,7560723,7593949,SSWWRHW,30/06/13,Cream Biscuits
3,8362837,15669764,SWLSCOZ,06/11/13,Besan
4,8202458,15670196,SSRCRSO,03/02/14,Organic F&V
...,...,...,...,...,...
5482,8269882,15668469,SWNHZNW,05/01/14,Beans
5483,8384422,15669875,SSWNRHC,18/11/13,Toor Dal
5484,7493590,15668465,SWRELHW,07/08/13,Root Vegetables
5485,8080319,15670267,SSNSECH,03/04/14,Toor Dal


In [11]:
order_member_sku_df = test_data[["Order", "Member", "SKU"]]
order_member_sku_df

,Order,Member,SKU
0,7409204,SWLCNOE,15669778
1,8076206,SWOEZES,15669977
2,7560723,SSWWRHW,7593949
3,8362837,SWLSCOZ,15669764
4,8202458,SSRCRSO,15670196
...,...,...,...
5482,8269882,SWNHZNW,15668469
5483,8384422,SSWNRHC,15669875
5484,7493590,SWRELHW,15668465
5485,8080319,SSNSECH,15670267


In [12]:
sku_freq_with_curr_order = pd.merge(sku_freq_df, order_member_sku_df, how = "left", on = ["Member", "SKU"])
sku_freq_with_curr_order

,Member,SKU,frequency,Order
0,SSCEHNS,7580823,7,NaN
1,SSCEHNS,15668377,7,NaN
2,SSCEHNS,15669865,7,8069966.0
3,SSCEHNS,15669772,6,NaN
4,SSCEHNS,15669970,6,8069966.0
...,...,...,...,...
18387,SWRNHCS,15669874,1,NaN
18388,SWRNHCS,15669886,1,NaN
18389,SWRNHCS,15670196,1,NaN
18390,SWRNHCS,15670260,1,NaN


In [13]:
SSCEHNS_past = sku_freq_df[sku_freq_df["Member"] == "SWOEZES"]

In [14]:
SSCEHNS_order = order_member_sku_df[(order_member_sku_df["Order"] == 8076206)]

In [15]:
recc_sku = list()
for past_sku in list(SSCEHNS_past["SKU"]):
    if past_sku in list(SSCEHNS_order["SKU"]):
        pass
    else:
        recc_sku.append(past_sku)

In [16]:
recc_sku[:5]

[15669869, 15669878, 15669866, 7587667, 7642810]

In [17]:
sku_freq_with_curr_order[(sku_freq_with_curr_order["Order"].isna()) & (sku_freq_with_curr_order["Member"] == "SWLCNOE")].head()

,Member,SKU,frequency,Order
10642,SWLCNOE,15668465,4,NaN
10644,SWLCNOE,15668459,3,NaN
10645,SWLCNOE,15668460,3,NaN
10646,SWLCNOE,15668467,3,NaN
10647,SWLCNOE,15668494,3,NaN


In [18]:
sku_freq_with_curr_order[sku_freq_with_curr_order["Member"] == "SWLCNOE"]

,Member,SKU,frequency,Order
10640,SWLCNOE,15668478,8,7409204.0
10641,SWLCNOE,15668379,6,7409204.0
10642,SWLCNOE,15668465,4,NaN
10643,SWLCNOE,15668457,3,7409204.0
10644,SWLCNOE,15668459,3,NaN
10645,SWLCNOE,15668460,3,NaN
10646,SWLCNOE,15668467,3,NaN
10647,SWLCNOE,15668494,3,NaN
10648,SWLCNOE,15668594,3,NaN
10649,SWLCNOE,15668688,3,7409204.0


In [19]:
recc_df_list = list()

for _, row in test_data[["Order", "Member"]].drop_duplicates().iterrows():
    df_temp = sku_freq_with_curr_order[sku_freq_with_curr_order["Member"] == row["Member"]].fillna("1000000000").sort_values(["Order", "frequency"], ascending = [False, False]).head()
    df_temp["Order"] = row["Order"]
    recc_df_list.append(df_temp.drop("frequency", axis = 1))

recc_df = pd.concat(recc_df_list, ignore_index = True)

In [20]:
recc_df = recc_df[["Order", "SKU", "Member"]].reset_index().rename(columns = {"index": "ID"})
recc_df["ID"] = recc_df["ID"].apply(lambda x: x+1)
recc_df

,ID,Order,SKU,Member
0,1,7409204,15668465,SWLCNOE
1,2,7409204,15668459,SWLCNOE
2,3,7409204,15668460,SWLCNOE
3,4,7409204,15668467,SWLCNOE
4,5,7409204,15668494,SWLCNOE
...,...,...,...,...
3185,3186,7682167,15668542,SWOSHSO
3186,3187,7682167,15669789,SWOSHSO
3187,3188,7682167,7586073,SWOSHSO
3188,3189,7682167,15669777,SWOSHSO


In [21]:
recc_df.to_csv("past_frequency_based_recc.csv", index = False)